# 1.1.0: Add microphones and listeners

## `MicArray` objects

`MicArray` objects are used in `AudibleLight` to represent various microphone array geometries, which may contain anywhere from between 1 to 64 individual microphone capsules arranged in a variety of different layouts. `MicArray` objects are `dataclasses`, defined in the Python standard library, and come packaged with numerous validation steps to ensure that they will work when added to a `Scene`.

## Import dependencies

In [2]:
from dataclasses import dataclass

import plotly.graph_objects as go
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots

from audiblelight import utils
from audiblelight.core import Scene
from audiblelight.micarrays import *

## Adding a single `MicArray`

To start, we'll add a simple tetra microphone (Sennheiser AmbeoVR) to a `Scene`.

In [2]:
scene = Scene(
    duration=60,
    sample_rate=44100,
    backend="rlr",
    state_kwargs=dict(
        mesh=utils.get_project_root() / "tests/test_resources/meshes/Oyens.glb"
    ),
    fg_path=utils.get_project_root() / "tests/test_resources/soundevents",
)

CreateContext: Context created


Microphones can be added with `Scene.add_microphone`. Here, we can specify the type of microphone, as well as its exact position within the mesh. Important also is the `alias` of the microphone, which provides a human-readable pointer to the microphone within the `Scene`.

Refer to the documentation for an overview of every parameter.

In [3]:
scene.add_microphone(
    microphone_type="ambeovr",    # or eigenmike32, eigenmike64, ...
    position=[-0.5, -0.5, 0.5],
    alias="my_first_mic"
)

CreateContext: Context created


In the above example, we provided coordinates for the center of the microphone in Cartesian format, with units given in meters. Now, we can print the coordinates of each capsule, to check this looks realistic.

In [4]:
placed = scene.get_microphone("my_first_mic")
print(placed.coordinates_absolute)

[[-0.49420772 -0.49420772  0.50573576]
 [-0.49420772 -0.50579228  0.49426424]
 [-0.50579228 -0.49420772  0.49426424]
 [-0.50579228 -0.50579228  0.50573576]]


## Adding multiple `MicArray`s

`AudibleLight` allows multiple microphones to be added to a `Scene`, with audio and metadata generated for each separately.

In [5]:
scene.clear_microphones()
scene.add_microphone(
    microphone_type="ambeovr",
    position=[-0.5, -0.5, 0.5],
    alias="my_first_mic"
)
scene.add_microphone(
    microphone_type="eigenmike32",
    alias="my_second_mic"
)
print(len(scene.state.microphones))

CreateContext: Context created


CreateContext: Context created
CreateContext: Context created
2


Now, we can add an `Event` and generate IRs for both microphones. IRs are provided in the shape `(n_capsules, n_emitters, n_samples)`.

In [6]:
scene.clear_events()
scene.add_event(event_type="static")
scene.state.simulate()

CreateContext: Context created


CreateContext: Context created


2025-10-20 09:19:10.766 | INFO     | audiblelight.core:add_event:839 - Event added successfully: Static 'Event' with alias 'event000', audio file '/home/huw-cheston/Documents/python_projects/AudibleLight/tests/test_resources/soundevents/music/007527.mp3' (unloaded, 0 augmentations), 1 emitter(s).


CreateContext: Context created


2025-10-20 09:19:10.998 | INFO     | audiblelight.worldstate:simulate:1743 - Starting simulation with 1 emitters, 2 microphones
Exception ignored in: <function BaseContext._insert_global.<locals>.on_disposal at 0x7b99dab97880>
Traceback (most recent call last):
  File "/home/huw-cheston/.cache/pypoetry/virtualenvs/audiblelight-5f5KpqNP-py3.10/lib/python3.10/site-packages/numba/core/typing/context.py", line 497, in on_disposal
    def on_disposal(wr, pop=self._globals.pop):
KeyboardInterrupt: 

KeyboardInterrupt



In [7]:
ambeo = scene.get_microphone("my_first_mic")
print(ambeo.irs.shape)

(4, 1, 72471)


In [8]:
eigen = scene.get_microphone("my_second_mic")
print(eigen.irs.shape)

(32, 1, 72471)


## Creating custom `MicArray`s

`AudibleLight` also defines a consistent API allowing new `MicArrays` to be defined easily within code. We can then add these to a `Scene`, generate impulse responses, and use these to convolve with existing audio files.

We can create a custom microphone array by superclassing `audiblelight.micarrays.MicArray`. We'll generate a custom microphone array that has a cuboid shape, containing eight capsules for the vertices of the cube.

In [9]:
@dataclass(eq=False)
class CubeMic(MicArray):
    name: str = "cube"
    is_spherical = False
    # layout can be either FOA or MIC
    channel_layout_type = "mic"

    @property
    def coordinates_polar(self) -> np.ndarray:
        """
        This property defines the polar coordinates of every capsule WRT the center.

        Azimuth is in range [-180, 180], increasing counter-clockwise
        Elevation is in range [-90, 90], where +0 == straight ahead
        Radius is given in metres and is unbounded.

        If you want, this information could also be provided in Cartesian format:
        just use `coordinates_cartesian` instead.
        """
        # Azimuth, elevation, radius
        return np.array(
            [
                [45, 90, 0.05],
                [135, 90, 0.05],
                [-135, 90, 0.05],
                [-45, 90, 0.05],
                [45, -90, 0.05],
                [135, -90, 0.05],
                [-135, -90, 0.05],
                [-45, -90, 0.05],
            ]
        )

    @property
    def coordinates_cartesian(self) -> np.ndarray:
        # Defined automatically, just included here for reference
        return utils.polar_to_cartesian(self.coordinates_polar)

    @property
    def capsule_names(self) -> list[str]:
        # Front-left upper, back-left upper, back-right upper, front-right upper
        # Front-left lower, back-left lower, back-right lower, front-right lower
        return ["FLU", "BLU", "BRU", "FRU", "FLL", "BLL", "BRL", "FRL"]

In [10]:
# Now, we add the cube mic to the scene in a random valid position
scene.clear_microphones()
scene.clear_events()
scene.add_microphone(microphone_type=CubeMic, alias="cuboid")
cube_mic_placed = scene.get_microphone("cuboid")

CreateContext: Context created
CreateContext: Context created


CreateContext: Context created


Now, we can add an event and simulate IRs for our custom microphone type

In [11]:
scene.add_event(event_type="static", duration=5)
scene.state.simulate()

2025-10-07 16:11:07.443 | INFO     | audiblelight.core:add_event:830 - Event added successfully: Static 'Event' with alias 'event000', audio file '/home/huw-cheston/Documents/python_projects/AudibleLight/tests/test_resources/soundevents/waterTap/95709.wav' (unloaded, 0 augmentations), 1 emitter(s).


CreateContext: Context created
CreateContext: Context created


2025-10-07 16:11:07.670 | INFO     | audiblelight.worldstate:simulate:1685 - Starting simulation with 1 emitters, 1 microphones
2025-10-07 16:11:18.756 | INFO     | audiblelight.worldstate:simulate:1693 - Finished simulation! Overall indirect ray efficiency: 0.991


In [12]:
# Access the IRs:
#  These have the shape (n_capsules, n_emitters, n_samples)
irs = cube_mic_placed.irs
print(irs.shape)

(8, 1, 72928)


## Visualise arrays

In this final cell, we'll visualise the capsule layouts of two microphones (AmbeoVR/Eigenmike32) using `plotly`.

In [8]:
arrays = [
    AmbeoVR(),
    Eigenmike32(),
    Eigenmike64()
]
fig = make_subplots(
    rows=1, 
    cols=len(arrays),
    specs=[
        [{"type": "scene"} for _ in range(len(arrays))],
    ],
    subplot_titles=[a.name for a in arrays]
)
fig.update_layout(
    width=360 * len(arrays),
    height=500,
    autosize=False,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=10,
    ),
)
for (mic_x, mic_y), mic_array in zip([(1, i + 1) for i in range(len(arrays))], arrays):
    df = pd.DataFrame(mic_array.coordinates_cartesian, columns=["x", "y", "z"])
    fig.add_trace(
        go.Scatter3d(
            x=df["x"],
            y=df["y"],
            z=df["z"],
            mode="markers",
            name=mic_array.name,
            marker_size=5,
            hovertext=mic_array.capsule_names
        ),
        row=mic_x,
        col=mic_y
    )
fig.show()